In [11]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt

In [12]:
#to do
###newdfclean
#Other data to get, public transportation, income by,  crime by 
#categorial: boro, tax code by class, zip-code,
# data= data[data['TAX_CLASS_AT_TIME_OF_SALE'] == 1 or 2]


In [93]:
data=pd.read_excel("rollingsales_manhattan.xls", skiprows=1) #combined excel sheets for 2016-2018

data = data[data['SALE_PRICE'] > 100] #get rid of 0's in sale price 
data=data[data['GROSS SQUARE FEET'] > 100] #get rid of 0s in square foot
data.columns=['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE']

In [121]:
# average_by_zip=pd.read_excel("Zip_MedianListingPrice_AllHomes.xls")
# average_by_zip_2018=average_by_zip.loc[:,["RegionName","2018-10"]] 
# average_by_zip_2018['ZIP_CODE']=average_by_zip_2018['RegionName']
# zillow_squarefootage['Zillowmedianlistprice']=zillow_squarefootage['2018-10']

zillow_squarefootage=pd.read_excel("Zip_MedianListingPricePerSqft_AllHomes.xls")
zillow_squarefootage=zillow_squarefootage.loc[:,["RegionName","2018-10"]] 
zillow_squarefootage['ZIP_CODE']=zillow_squarefootage['RegionName']
zillow_squarefootage['ZillowSquareFootage']=zillow_squarefootage['2018-10']

zillow_median_listing=pd.read_excel("Zip_MedianListingPrice_AllHomes.xls")
zillow_median_listing=zillow_median_listing.loc[:,["RegionName","2018-10"]] 
zillow_median_listing['ZIP_CODE']=zillow_median_listing['RegionName']
zillow_median_listing['ZillowMedianPrice']=zillow_median_listing['2018-10']

In [126]:
new_df2= zillow_squarefootage.merge(zillow_median_listing, how = 'inner', on = ['ZIP_CODE'])
# new_df3=new_df2.merge(average_by_zip_2018, how = 'inner', on = ['ZIP_CODE'])
new_df3= data.merge(new_df2, how = 'inner', on = ['ZIP_CODE'])
new_df=new_df3.copy()
new_df=new_df.drop(columns=['RegionName_y', 'RegionName_x',"2018-10_x","2018-10_y"])
new_df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE', 'ZillowSquareFootage', 'ZillowMedianPrice'],
      dtype='object')

In [127]:
#Changing the datatype of Median Price and Square Footage from Float to Integer
new_df['ZillowMedianPrice'] = new_df['ZillowMedianPrice'].astype(int)
new_df['ZillowSquareFootage'] = new_df['ZillowSquareFootage'].astype(int)
target=new_df[["SALE_PRICE"]]
features= new_df 
features=features.drop(columns=['BOROUGH',                     
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 
        'TOTAL_UNITS',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE','ZillowSquareFootage', 'ZillowMedianPrice',
      'SALE_PRICE']) #maybe drop zillow two 
features["BUILDING_CLASS_CATEGORY"]=features["BUILDING_CLASS_CATEGORY"].str.strip()
features["BUILDING_CLASS_CATEGORY"]=features["BUILDING_CLASS_CATEGORY"].str.replace(' ', '')


In [128]:
# print(len(features.BUILDING_CLASS_CATEGORY.unique()))
# cat_vars.columns
# features
features['SALE_DATE'].iloc[0]
features['SALE_DATE'] = pd.to_datetime(features['SALE_DATE'])
features['SALE MONTH'] = features['SALE_DATE'].dt.month
features.drop(columns='SALE_DATE', inplace=True)
cat_vars=features[['BUILDING_CLASS_CATEGORY','ZIP_CODE',"NEIGHBORHOOD",'SALE MONTH']]

In [129]:

for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(features[var], prefix=var,drop_first=True)
    data1=features.join(cat_list)
    features=data1
data_vars=features.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
features=features[to_keep]


In [132]:
features.columns
# for i in features.columns:
#     print(i)
len(features.columns)

96

## To do
- get the features listed below
- plot the different square feet things together to see what the relationship is
    - land square feet
    - gross square feet
    - zillow square feet
- try the regression with and without zillow median price
- clean up building_class_category
- run regression
- try feature engineering/scaling

## Model

y - sale price

### Features 
- residential units (done)
- commercial units
- land_square_feet
- gross_square_feet
- zillow square feet
- year_built
- building_class_category (CATEGORY)
- neighborhood OR zip code (CATEGORY)
- time - optional (CATEGORY - grab either month or year)

In [133]:
features.columns[0:8]
len(features.columns)

96

In [134]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics

# category_features=features[features.columns.difference(['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
#        'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
#         'ZillowSquareFootage', 'ZillowMedianPrice'])] #[84:106]

# scaled_features=features[['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
#        'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
#         'ZillowSquareFootage', 'ZillowMedianPrice']]
# len(category_features.columns)
                

In [135]:
# REGRESSION STUFF BELOW 
# REGRESSION STUFF BELOW 
# REGRESSION STUFF BELOW 
# REGRESSION STUFF BELOW 

In [136]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [137]:
train1= features # can change to scaled_features or features to test regresion model with or without categorical values 
labels=target
# train1

In [138]:
x_train , x_test , y_train , y_test = train_test_split(train1 , labels , test_size = 0.20,random_state =5)

In [139]:
reg.fit(x_train,y_train)
reg.score(x_test,y_test) 
#highest score with all variables (2110) is .79

0.7386501639917878

In [57]:
x_train

RESIDENTIAL_UNITS  COMMERCIAL_UNITS  LAND_SQUARE_FEET  GROSS_SQUARE_FEET  \
77                  23                 4              5042              25248   
427                256                 2             51180             262875   
1                    4                 6              3788              18500   
175                  0                 2              7532             112850   
356                  0                 2              7532             112850   
340                  0                 2              7532             112850   
384                  0                 2              7532             112850   
565                  2                 1              1892               3312   
773                  6                 0              1998               3025   
789                  0                 1              2525               8990   
116                  0                 2              7532             112850   
470                  0                 1              3372               5930   
524                 12                 0              2469               7080   
569                 34                 6              4837              24170   
174                  0                 2              7532             112850   
767                 11                 0              1499               2286   
846                476                 6             17623             400531   
71                  20                 0              2510               9695   
286                  0                 2              7532             112850   
826                 24                 0              4489              18523   
119                  0                 2              7532             112850   
177                  0                 2              7532             112850   
685                  0                11             11067              47621   
305                  0                 2              7532             112850   
13                   0                 4              1480               5495   
798                 12                 0              2370              10715   
643                  8                 3              1762               7710   
595                 28                 3              4037              17156   
644                  1                 0              1356               6250   
696                 44                 1             14350              78397   
..                 ...               ...               ...                ...   
550                  7                 1              1618               7590   
359                  0                 2              7532             112850   
51                   6                 1              1008               5040   
772                 11                 0              1749               2625   
102                  0                 2              7532             112850   
195                  0                 2              7532             112850   
679                  2                 1              1650               3468   
805                 11                 1              2388               8045   
628                  0                 6              5995              27000   
124                  0                 2              7532             112850   
404                  0                61             19700             327766   
730                  3                 0              1817               2827   
287                  0                 2              7532             112850   
831                  5                 1              1779               3713   
47                   0                 1              2500               7500   
725                  6                 0              2018               3354   
587                  4                 0              1766               2412   
607                  8              

In [ ]:
from sklearn.linear_model import Ridge
from sklearn import metrics
ridgeReg = Ridge(alpha=5, normalize=True)

ridgeReg.fit(x_train,y_train)

y_pred = ridgeReg.predict(x_test)
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# old notes 
# data_final=features[to_keep]
# len(data_final.columns)
# # data_final

#other 


#####used for test below 
# scaled_features=features[[
#         'GROSS_SQUARE_FEET' 
#         ]]
# scaled_features=features[[ 'TOTAL_UNITS',    # got a score of .71 with this 
#         'GROSS_SQUARE_FEET', 'YEAR_BUILT',
#         'ZillowSquareFootage', 'ZillowMedianPrice']]

# scaled_features=features[[ 'TOTAL_UNITS', # score of .71 
#         'GROSS_SQUARE_FEET', 'YEAR_BUILT',
#         'ZillowSquareFootage', 'ZillowMedianPrice']]
# scaled_features.info()
# # scaled_features
# cat_vars=features[["ZIP_CODE",'BOROUGH',"NEIGHBORHOOD","BUILDING_CLASS_CATEGORY","TAX_CLASS_AT_PRESENT"]].copy()
# features    
##### test for using one featuer below 
# scaler.fit(data_final.iloc[:,:8])
# data_final.columns.values
# scaler.fit(data_final.iloc[:,:8])
# print(len(category_features.columns))
# print(len(scaled_features.columns))
# print(len(features.columns))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import matplotlib
matplotlib.rcParams.update({'font.size': 14})
from sklearn.datasets import load_boston
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
# ridgeReg = Ridge(alpha=5, normalize=True)

# ridgeReg.fit(X_train,y_train)

# y_pred = ridgeReg.predict(X_test)

In [ ]:
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# print(np.sqrt(metrics.mean_squared_error(y_test, y_pred))/ y_test.outcome.std())

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
reg.fit(x_train,y_train)
reg.score(x_test,y_test)

In [ ]:
from sklearn import preprocessing
from sklearn import pipeline

scaler = preprocessing.StandardScaler()

In [ ]:
scaler.fit(features.iloc[:,:])

In [ ]:
scaler.fit(X_train.iloc[:,:])
features_scaled_train = pd.DataFrame(scaler.transform(X_train.iloc[:,:]), columns=X_train.columns[:], index=X_train.index)
features_scaled_train['LAND_SQUARE_FEET']= X_train.LAND_SQUARE_FEET

features_scaled_train.head()

In [ ]:
features_scaled_test = pd.DataFrame(scaler.transform(X_test.iloc[:,:]), columns=X_test.columns[:], index=X_test.index)

features_scaled_test.head()

In [ ]:
poly = preprocessing.PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
features_64_train = pd.DataFrame(poly.fit_transform(features_scaled_train), columns=poly.get_feature_names(features_scaled_train.columns))
features_64_train.head() #making more features 

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
features_64_train.head()

In [ ]:
features_64_test = pd.DataFrame(poly.fit_transform(features_scaled_test), columns=poly.get_feature_names(features_scaled_test.columns))
features_64_test.head()

In [ ]:
from sklearn.feature_selection import VarianceThreshold


In [ ]:
thresholder = VarianceThreshold(threshold=.5)

def variance_threshold_selector(features, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(features)
    return features[features.columns[selector.get_support(indices=True)]]

In [ ]:
features_selected_train = variance_threshold_selector(features_64_train)

In [ ]:
features_selected_train.head()


In [ ]:
import seaborn as sns

sns.set(style="white")


# Compute the correlation matrix
corr = features_selected_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# Create correlation matrix
corr_matrix = features_selected_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [ ]:
features_selected_train.drop(columns=to_drop, inplace=True)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression


In [ ]:
def information_selector(X, y, scoring, k=5):
    selector = SelectKBest(score_func=scoring, k=k)
    selector.fit(X, y)
    return X[X.columns[selector.get_support(indices=True)]]

In [ ]:
test = SelectKBest(score_func=mutual_info_regression, k=30) #retunrs top 30 variables
fit = test.fit(features_selected_train, y_train)

In [ ]:
features_selected_train[features_selected_train.columns[fit.get_support(indices=True)]].head()

In [ ]:
# fit a model
lm = linear_model.LinearRegression()
model = lm.fit(features_selected_train, y_train)

In [ ]:
features_selected_test = features_64_test[features_selected_train.columns]
y_pred = lm.predict(features_selected_test)

print(metrics.mean_absolute_error(y_test, y_pred))
print(metrics.mean_squared_error(y_test, y_pred))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:

# for var in cat_vars.ZIP_CODE:
#     cat_list='var'+'_'+var
#     cat_list = pd.get_dummies(new_df[var])
#     data1=new_df.join(cat_list)
#     new_df_final=data1
# data_vars=new_df.columns.values.tolist()
# to_keep=[i for i in data_vars if i not in cat_vars]

In [ ]:
plt.scatter(new_df.ZIP_CODE,new_df.SALE_PRICE)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

In [ ]:
#create dummy variable to deal with categories... can make dummies 


#NExt test train split  Example
# create training and testing vars
# X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=34,test_size=0.2)
# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)


## fit a model
# lm = linear_model.LinearRegression()
# model = lm.fit(X_train, y_train)
# y_pred = lm.predict(X_test)

# ## The line / model
# plt.scatter(y_test, y_pred)
# plt.xlabel("True Values")
# plt.ylabel("Predictions")

##Print R Squared score
# print ("Score:", model.score(X_test, y_test))
##Score: 0.5496539094948476

##average of metrics 
# print(metrics.mean_absolute_error(y_test, y_pred))
# print(metrics.mean_squared_error(y_test, y_pred))
# print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

model.score()



In [ ]:
for column in features:
    data[column].plot.hist(density=True, label = column+' histogram')
    data[column].plot.kde(label =column+' kde')
    plt.legend()
    plt.show()

In [ ]:
[:,'BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
#        'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
#        'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
#        'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
#        'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
#        'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
#        'SALE_PRICE', 'SALE_DATE','Zillowsquarefootagezillow','zillowMedianPrice']

In [ ]:
# import seaborn as sns
# import statsmodels.api as sm
# import pylab
# sns.pairplot(data)[]

In [ ]:
# data.info()
# data.dropna()
# data['SALE_PRICE'] = data['SALE_PRICE']
# carl=data['SALE_PRICE']
# # data['SALE_PRICE']
# # data
# data = data.drop(0,axis=0)
# data['GROSS SQUARE FEET'] > 0]   #830 entries
# data[data['LAND SQUARE FEET'] > 0] # 863 entries 
# data.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
plt.style.use('fivethirtyeight')

data = data
f = 'SALE_PRICE~GROSS_SQUARE_FEET'
f2='SALE_PRICE~NEIGHBORHOOD'
f3='SALE_PRICE~LAND_SQUARE_FEET'
f4='SALE_PRICE~YEAR_BUILT'
f5= 'SALE_PRICE~ZIP_CODE'
model = smf.ols(formula=f, data=data).fit()
model2 = smf.ols(formula=f2, data=data).fit()
model3 = smf.ols(formula=f3, data=data).fit()
model4 = smf.ols(formula=f4, data=data).fit()
model5 = smf.ols(formula=f5, data=data).fit()
# model.summary()
# model.params
# model.pvalues
# model.rsquared

In [ ]:
#look at each distribtions, graph them all 

In [ ]:
# data.plot.kde()
# plt.title("Normality check")
# plt.show()
# fig = plt.figure(figsize=(15,8))
# fig = sm.graphics.plot_regress_exog(model, "GROSS_SQUARE_FEET", fig=fig)
# plt.show()
data['GROSS_SQUARE_FEET'].plot.hist(normed=True, label =' histogram')
data['GROSS_SQUARE_FEET'].plot.kde(label ='kde')
# plt.legend()
# plt.show()

In [ ]:
data['SALE_PRICE'].plot.hist(normed=True, label =' histogram')
data['SALE_PRICE'].plot.kde(label ='kde')
# plt.legend()
# plt.show()

In [ ]:
import scipy.stats as stats
residuals = model.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
fig.show()

In [ ]:
model.summary()

In [ ]:
model.summary() #y variale is 799.36

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model3, "LAND_SQUARE_FEET", fig=fig)
plt.show()
import scipy.stats as stats
residuals = model3.resid
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
fig.show()

In [ ]:
model4.summary()

In [ ]:
model5.summary()

In [ ]:
# #first we need to import some libraries
# import pylab
# import pandas as pd
# # import pandas.io.data as web
# import datetime
# %matplotlib inline
# import matplotlib.pyplot as plt
# from matplotlib import style
# import numpy as np

# import quandl
# #ZILLOW/{AREA_CATEGORY}{AREA_CODE}_{INDICATOR_CODE}
# #ZILLOW/C1_ZRISFRR

In [ ]:
# quandl.ApiConfig.api_key = 'DoG1Rex2WTP5SQt6mchR'
# # quandl.get('ZILLOW/C25709_ZRISFRR', start_date='2018-10-31', end_date='2018-10-31')
# quandl.get('ZILLOW/C1_IMP')

# quandl.get('ZILLOW/C1_ZRISFRR', start_date='2018-10-31', end_date='2018-10-31')

In [ ]:
#quandl tries below can delete later

In [ ]:
# quandl.ApiConfig.api_key = 'DoG1Rex2WTP5SQt6mchR'
# quandl.get('ZILLOW/C1_IMP', start_date='2018-08-31', end_date='2018-10-31')

In [ ]:
# quandl.ApiConfig.api_key = 'DoG1Rex2WTP5SQt6mchR'
# quandl.get('ZILLOW/C1_IMP')

In [ ]:
# quandl.ApiConfig.api_key = 'DoG1Rex2WTP5SQt6mchR'
# quandl.get('ZILLOW/C1_IMP')